In [1]:
import numpy as np

We note that we are using the base field `QQ` for all computations currently

Let us fix $N$, $d$.

Let `R`$ := \mathbb{C}[y_{11}, y_{12}, y_{21}, y_{22}]$ and let `R^d`$ := \mathbb{C}[y_{11}, y_{12}, y_{21}, y_{22}]^d$.
<br>
Let `V`$ := \mathbb{C}^{{N+d-1}\choose{d}} \cong  \mathbb{C}[y_{11}, y_{12}, y_{21}, y_{22}]^d$
<br>
Let `glN` $:= \mathfrak{gl}_N \cong M_{N}$.

In [2]:
n = 4
N = 16
d = 4
indeterminates = [var("y_%d%d" % ((i+1),(j+1))) for i in range(n) for j in range(n)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

V = VectorSpace(QQ, binomial(N + d - 1, d))
glN = MatrixSpace(QQ, N)

Defining y_11, y_12, y_13, y_14, y_21, y_22, y_23, y_24, y_31, y_32, y_33, y_34, y_41, y_42, y_43, y_44


In [3]:
M = MatrixSpace(R, 4)
A = M(np.array(indeterminates).reshape(-1,4))
print(A)
f = [R(A.determinant())]

[y_11 y_12 y_13 y_14]
[y_21 y_22 y_23 y_24]
[y_31 y_32 y_33 y_34]
[y_41 y_42 y_43 y_44]


In [4]:
# (glN, R^d) -> R^d
def X_action(X, p):
    grad_p = p.gradient()
    sum = 0
    for a in range(N):
        for b in range(N):
            sum += -X[a][b]*indeterminates[b]*grad_p[a]
    return R(sum)

In [5]:
## gives "the" ordering of the degree d monomials in R to allow for a natural correspondence with coefficient vectors
symmfuncs = SymmetricFunctions(QQ).h()
monomials = symmfuncs([d]).expand(N, alphabet=indeterminates).monomials()

# R^d -> V
def polynomial_to_vector(p):
    if not p in R:
        return 0
    coefficient_list = []
    for i in range(len(monomials)):
        coefficient_list.append(p.monomial_coefficient(monomials[i]))
    return V(coefficient_list)

# V -> R^d
def vector_to_polynomial(v):
    if not v in V:
        return 0
    p = 0
    for i, item in enumerate(Sequence(monomials)):
        p += v[i]*item
    return R(p)

In [37]:
# W = span{f}
W = V.subspace([polynomial_to_vector(fi) for fi in f])
W_perp = W.complement()

print('W:', W)
print('W_perp:', W_perp)

g = W_perp.basis()

p = W.dimension()
q = W_perp.dimension()

W: Vector space of degree 3876 and dimension 1 over Rational Field
Basis matrix:
1 x 3876 dense matrix over Rational Field
W_perp: Vector space of degree 3876 and dimension 3875 over Rational Field
Basis matrix:
3875 x 3876 dense matrix over Rational Field


Note `M_f` $:= \tilde{M}_f$

In [38]:
def sparsify(v):
    flattened_list = []
    for index, value in enumerate(v):
        if v[index] != 0:
            flattened_list.append((index,value))
    return flattened_list

def sparse_inner_product(a,b):
    i = 0; j = 0
    result = 0
    while i < len(a) and j < len(b):
        if a[i][0] == b[j][0]:
            result += a[i][1]*b[j][1]
            i += 1
            j += 1
        elif a[i][0] < b[j][0]:
            i += 1
        else:
            j += 1
    return result

In [42]:
E = lambda i,j: list(glN.basis())[(N*i) + j] # returns E_ij basis of glN

ActionMemo = [[[sparsify(polynomial_to_vector(X_action(E(i,j),f[l]))) for i in range(N)] for j in range(N)] for l in range(p)]
g_sparse = [sparsify(g[i]) for i in range(len(g))]

In [44]:

# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(p*l) + k,(N*i) + j] = sparse_inner_product(ActionMemo[l][i][j], g_sparse[k])
                
print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())
print(2*(n^2)-1)

dim(ker(M_f)) = 31
31
